# The client first request the public key of asymmetric encryption from the server
# Client generates a key for symmetric encryption
# Client uses public key obtained from server to encrypt the public key and send to server
# Server uses its own private key to decrypted the received the encrypted key from client
# After the above negotiation process, server and client will use use symmetric encryption to encrypt the following communication with the negotiated key

In [56]:
import socket
import secrets

ModuleNotFoundError: No module named 'cryptography'

In [18]:
# define a special request
class Request:
    # a simple request class containing method, url and headers
    def __init__(self, method: str, url: str, headers: {str: str}, request_data: str, body:str):
        self.method = method  # GET, POST, PUT, DELETE
        self.url = url  # e.g. /index.html
        self.headers = headers
        self.request_data = request_data
        self.body = body

    def __repr__(self) -> str:
        # output request
        # custom ENCRYPTION/1.1 request
        request = "{} {} ENCRYPTION/1.1\r\n".format(self.method, self.url)
        request += "\r\n".join("{}: {}".format(k, v) for k, v in self.headers.items())
        request += "\r\n\r\n"
        # request += self.request_data.split("\r\n\r\n", 1)[1]  # Append file data
        return request

In [40]:
# socket connect 127.0.0.1:8080/
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect(("127.0.0.1", 8080))

In [52]:
# first step: client request a public key
request = Request("GET", "/public_key", {}, "", "")
s.sendall(repr(request).encode("utf-8"))
while True:
    r = s.recv(1024)
    if r is not None:
        break
public_key = r.decode('utf-8').split(',')[0].split('(')[1].strip(" ")
print("Public Key:", public_key)

Public Key 16929592767465609893667197996842639979939424032098774800139278703927488126983862324777431405113654797272608809110123832808707891077618124697398850668262707393164835093591674010260276462728548392922787414053906722727761340786402330177631595691000361205780030329466241611273008566633521841336332655654759771387860538625925102175686901901573335423221702859259615561899179873614976512247316703722027712970372864406617341662930481787455276397012433563337078234646314903551076942146764784396826708171560827907786339644507870302725540734623984121888315074450812808067823747891681768782621305337819100763606271126079656753253


In [53]:
# second step: Client generates a key for symmetric encryption
key = secrets.token_bytes(32)
print('Key Generated', key)

b'\x16\xc0`\x81\x1ct<\x10\xc9n\x8b\\\x7f\xde\xc4\xcdk\xb4\x9d&\xf4\xfa\x91\xd9\xfa\xa9\xef\x91\xa1\x07\xf7c'
